In [1]:
import numpy as np
import heapq as hq
from copy import deepcopy
filename = "input_16"
#filename = "example_16.txt"
#filename = "example_16_2.txt"
with open(filename) as f:
    input = [[i for i in line[1:-1]] for line in f.read().split("\n")[1:-1]]

In [ ]:
def print_grid(grid):
    for line in grid:
        print("".join(line))
#print_grid(input)

In [3]:
def find_shortest_path(input):

    xlen = len(input[0])
    ylen = len(input)
    direction_map = {0:(0,1), 1:(0,-1), 2:(1,0), 3:(-1,0)}
    allowed_turns = {0:[2,3], 1:[2,3], 2:[0,1], 3:[0,1]}
    grid = np.zeros((4,ylen,xlen), dtype="O")
    c = 0 # use this counter to remove ambiguities in heap ordering

    # prepare input:
    i0, j0 = tuple(i[0] for i in np.where(np.array(input) == "S")) #starting
    iN, jN = tuple(i[0] for i in np.where(np.array(input) == "E")) #end
    walls = np.array(np.where(np.array(input) == "#")).T.tolist()
    for i in range(xlen):
        for j in range(ylen):
            for d in range(4):
                grid[d][i][j] = {
                    "position": (i,j,d),
                    "distance": float("inf"),
                    "predecessor": [],
                }
    grid[0][i0][j0]["distance"] = 0
    heap = [(0, 0, grid[0][i0][j0])]
    seen = set()

    while heap:
        _, _, current_node = hq.heappop(heap)
        current_position = current_node["position"]
        i, j, d = current_position
        if (i,j) == (iN, jN): break
        if current_position in seen: continue
        seen.add(current_position)
  
        # can either change direction...
        for new_d in allowed_turns[d]:
            if (i, j, new_d) not in current_node["predecessor"]:
                node = grid[new_d][i][j]
                dist = current_node["distance"] + 1000
                if dist < node["distance"]:
                    grid[new_d][i][j]["distance"] = dist
                    grid[new_d][i][j]["predecessor"] = [current_position]
                    hq.heappush(heap, (dist, (c:=c+1), grid[new_d][i][j]))
                elif dist == node["distance"]:
                    grid[new_d][i][j]["predecessor"].append(current_position)
        
        # ... or go into the current direction:
        current_dir = direction_map[d]
        y = i + current_dir[0]
        x = j + current_dir[1]
        if y in range(ylen) and x in range(xlen) and [y,x] not in walls:
            node = grid[d][y][x]
            dist = current_node["distance"] + 1
            if dist < node["distance"]:
                grid[d][y][x]["distance"] = dist
                grid[d][y][x]["predecessor"] = [current_position]
                hq.heappush(heap, (dist, (c:=c+1), grid[d][y][x]))
            elif dist == node["distance"]:
                grid[d][y][x]["predecessor"].append(current_position)

    return grid

In [ ]:
# part 1:
grid = deepcopy(input)
nodes = find_shortest_path(grid)
print(min([nodes[i][0][-1]["distance"] for i in range(4)]))

In [13]:
# part 2:
def find_best_tiles(nodes, input):
    best_tiles = set()
    iN, jN = tuple(i[0] for i in np.where(np.array(input) == "E")) #end
    dN = np.argmin([nodes[i][0][-1]["distance"] for i in range(4)]) # final orientation
    tiles_to_check = [(iN, jN, dN)]
    while tiles_to_check:
        i, j, d = tiles_to_check.pop()
        best_tiles.add((i,j))        
        for next_tile in nodes[d][i][j]["predecessor"]:
            tiles_to_check.append(next_tile)
    return len(best_tiles), best_tiles

In [ ]:
number_of_best_tiles, best_tiles = find_best_tiles(nodes, input)
print(number_of_best_tiles)

In [ ]:
def show_best_paths(input, best_tiles):
    for i,j in best_tiles:
        input[i][j] = "O"
    for line in input:
        print("".join(line))
#show_best_paths(input, best_tiles)